# Sensor/Pump Models and Human-in-the-loop

Simulate sensor noise, pump caps, and real-time interventions.


In [ ]:
import json
from pathlib import Path

import iints
import yaml
from iints.core.algorithms.pid_controller import PIDController
from iints.core.devices import SensorModel, PumpModel
from iints.core.simulator import Simulator
from iints.core.patient.models import PatientModel
from iints.validation import build_stress_events

presets = json.loads(Path("src/iints/presets/presets.json").read_text())
preset = next(p for p in presets if p["name"] == "midnight_crash")

patient_config = yaml.safe_load(Path("src/iints/data/virtual_patients/{}".format(preset["patient_config"] + ".yaml")).read_text())
patient_model = PatientModel(**iints.validation.validate_patient_config_dict(patient_config).model_dump())

sensor = SensorModel(noise_std=3.0, bias=1.0, lag_minutes=5, dropout_prob=0.05, seed=1)
pump = PumpModel(max_units_per_step=0.5, quantization_units=0.05, dropout_prob=0.02, seed=1)

def on_step(ctx):
    # If glucose is too low, inject 12g rescue carbs
    if ctx["glucose_actual_mgdl"] < 70:
        return {"additional_carbs": 12, "note": "rescue carbs"}
    return None

simulator = Simulator(
    patient_model=patient_model,
    algorithm=PIDController(),
    time_step=preset["time_step_minutes"],
    sensor_model=sensor,
    pump_model=pump,
    on_step=on_step,
    seed=42,
)

for event in build_stress_events(preset["scenario"]["stress_events"]):
    simulator.add_stress_event(event)

results, safety = simulator.run_batch(duration_minutes=240)
results[["time_minutes", "glucose_actual_mgdl", "sensor_status", "pump_status", "human_intervention"]].head()
